# Setup do Ambiente

In [ ]:
!pip install -q unsloth[colab-new] faiss-cpu sentence-transformers datasets accelerate trl peft transformers
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

import json
import re
import os
import unicodedata
import faiss
import numpy as np
import torch

from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import pipeline
from transformers import TrainingArguments
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from datasets import Dataset
from trl import SFTTrainer
from dotenv import load_dotenv
from huggingface_hub import login


from google.colab import drive
drive.mount('/content/drive')

# Download e exploração inicial dos dados

In [ ]:
!git clone https://github.com/pubmedqa/pubmedqa.git

file_path = 'pubmedqa/data/ori_pqal.json'

with open(file_path, 'r') as f:
    data = json.load(f)

sample_key = list(data.keys())[0]
print(f"\nCampos disponíveis: {list(data[sample_key].keys())}\n")

print("=" * 60)
print("Exploração de dados - PubMedQA")
print("=" * 60)

for i, key in enumerate(list(data.keys())[:3]):
    item = data[key]

    print(f"\nExemplo {i+1} | ID: {key}")
    print("-" * 60)
    print(f"Question: {item.get('QUESTION', 'N/A')}")

    context = " ".join(item.get('CONTEXTS', []))
    print(f"Context: {context[:300]}...")

    print(f"Labels: {item.get('LABELS', 'N/A')}")
    print(f"Decision: {item.get('final_decision', 'N/A')}")
    print(f"Answer: {item.get('LONG_ANSWER', 'N/A')[:200]}...")
    print(f"Meshes: {item.get('MESHES', 'N/A')}")
    print(f"Year: {item.get('YEAR', 'N/A')}")
    print(f"Reasoning required pred: {item.get('reasoning_required_pred', 'N/A')}")
    print(f"Reasoning free pred: {item.get('reasoning_free_pred', 'N/A')}")

print(f"\n\nTotal de registros: {len(data)}")

# Pré-processamento + Anonimização

In [ ]:
MAP_DECISION = {
  "yes": "SIM",
  "no": "NÃO",
  "maybe": "TALVEZ"
}


def anonymize_text(text):
    """Remove dados sensíveis (LGPD/HIPAA compliance)"""
    if not isinstance(text, str):
        return ""

    text = re.sub(r'(Dr\.|Dra\.|Doctor|Prof\.|MD)\s+[A-Z][a-z]+(\s+[A-Z][a-z]+)?', '[NOME]', text)
    text = re.sub(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '[EMAIL]', text)
    locations = r'(Israel|Denmark|Chile|Texas|France|United Kingdom|UK|USA|Pakistan|Karachi|Jordan|Japan|Australia|North Carolina|Washington)'
    text = re.sub(locations, '[LOCAL]', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\d{6,}\b', '[ID]', text)
    text = re.sub(r'\b(19|20)\d{2}\b', '[ANO]', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '[URL]', text)
    return text

def clean_text(text):
  if not text:
    return ""
  text = unicodedata.normalize("NFKC", text)
  text = re.sub(r"\s+", " ", text).strip()

  text = anonymize_text(text)

  return text


processed_docs = []

for item in data.values():
  question = clean_text(item.get("QUESTION"))
  context = clean_text(" ".join(item.get("CONTEXTS", [])))
  answer = clean_text(item.get("LONG_ANSWER", ""))
  decision = MAP_DECISION.get(item.get("final_decision", ""), "")


  if question and context:
    processed_docs.append({
      "question": question,
      "context": context,
      "answer": answer,
      "decision": decision,
      "year": item.get("YEAR")
    })


print(len(processed_docs))

# Database em português para treinamento do modelo

In [ ]:
dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/rag/language_alignment_pt.json",
    split="train"
)

print(dataset.column_names)

def messages_to_text(example):
    text = ""
    for msg in example["messages"]:
        if msg["role"] == "system":
            text += f"<<SYS>>\n{msg['content']}\n<</SYS>>\n\n"
        elif msg["role"] == "user":
            text += f"[INST] {msg['content']} [/INST]\n"
        elif msg["role"] == "assistant":
            text += msg["content"]
    return {"text": text}

dataset = dataset.map(
    messages_to_text,
    batched=False,
    remove_columns=["messages"]
)

print(dataset.column_names)
print(dataset[0])

# Preparação para RAG Médico

In [ ]:
rag_documents = []

for d in processed_docs:
  text = f"""
  Pergunta científica:
  {d['question']}


  Evidência:
  {d['context']}


  Conclusão:
  {d['answer']}
  """
  rag_documents.append(text.strip())


len(rag_documents)

#Criação de Embeddings + Índice FAISS

In [ ]:
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedder.encode(rag_documents, show_progress_bar=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

# Create the directory if it doesn't exist
os.makedirs('/content/drive/MyDrive/rag', exist_ok=True)

faiss.write_index(index, "/content/drive/MyDrive/rag/medical_index.faiss")

with open('/content/drive/MyDrive/rag/medical_docs.json', 'w') as f:
  json.dump(rag_documents, f)

## Fine Tunning LoRA

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    device_map="auto"
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0.05,
)

# Treinamento do modelo para respostas em Português

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    learning_rate=2e-4,
    logging_steps=5,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=training_args,
    dataset_text_field="text",
    max_seq_length=512,
    packing=False
)

trainer.train()

# Salvando o modelo no HuggingFace

In [ ]:
# Carregar env
ENV_PATH = "/content/drive/MyDrive/token-hf/env"
load_dotenv(ENV_PATH)

HF_TOKEN = os.getenv("HF_TOKEN")
login(token=HF_TOKEN)

HF_REPO = f"{os.getenv("HF_USER_REPO")}/assistente-medico-lora"
model.push_to_hub(HF_REPO)
tokenizer.push_to_hub(HF_REPO)

# Preparação da llm com Prompt

In [ ]:
SYSTEM_PROMPT = """
Você é um assistente médico virtual.
Responda sempre em português, com linguagem clara, empática e baseada em evidências científicas.


Regras:
- Não invente informações.
- Se não houver evidência suficiente, diga isso explicitamente.
- Não prescreva medicamentos nem indique tratamentos específicos.
- Não indique remédios ou tratamentos.
- Quando perguntarem por algum remédio, você deve responder: Não estou autorizado a prescrever medicamentos, por favor, consulte um médico.
- Sempre cite a fonte da informação científica
"""

with open('/content/drive/MyDrive/rag/medical_docs.json') as f:
  docs = json.load(f)

index = faiss.read_index('/content/drive/MyDrive/rag/medical_index.faiss')

llm = pipeline(
  "text-generation",
  model=model,
  tokenizer=tokenizer,
  max_new_tokens=300,
  temperature=0.0,
  do_sample=False,
  repetition_penalty=1.1,
  return_full_text=False,
  eos_token_id=tokenizer.eos_token_id
  )

def retrieve_context(question, k=3):
  q_emb = embedder.encode([question])
  _, idx = index.search(q_emb, k)
  return "\n\n".join([docs[i] for i in idx[0]])

def medical_chat(question):
  context = retrieve_context(question)

  prompt = f"""
{SYSTEM_PROMPT}


Contexto científico relevante:
{context}


Pergunta: {question}
Resposta:
"""
  return llm(prompt)[0]['generated_text']

# Teste Básico do Modelo


In [ ]:
question = "O que a literatura indica sobre o uso de aspirina em prevenção primária?"
print(medical_chat(question))

In [ ]:
question = "Qual medicamento é eficaz para pedra nos rins?"
print(medical_chat(question))